[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/galilee.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

! pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@2c0d7859a30cc97d8cd18139655c54c97cdb2615" 
import generic_solver


In [ ]:
! wget https://github.com/GP211/2023-fall-class-notebooks/raw/main/data/galilee.H

In [ ]:
from sep_python import default_io
vec=default_io.vector_from_storage("./galilee.H")

In [ ]:
print(vec)

In [ ]:
import copy
from generic_solver._pyOperator import Operator
class Bin2D(Operator):

    def __init__(self, mod, dat, xy):
        """
        Initialize the binning operator.
        """
        super().__init__(mod, dat)
        hyper = mod.get_hyper()
        
        ax0 = hyper.axes[0]
        n1, o1, d1 = ax0.n, ax0.o, ax0.d
        
        ax1 = hyper.axes[1]
        n2, o2, d2 = ax1.n, ax1.o, ax1.d
        
        num_points = xy.shape[1]
        self._i1 = np.zeros(num_points, dtype=np.int32)
        self._i2 = np.zeros(num_points, dtype=np.int32)
        self._sc = np.ones(num_points)
        
        for x_val, y_val, index, in zip(xy[0], xy[1],range(num_points)):
            self._i1[index] = (x_val - o1) / d1 + 0.5
            self._i2[index] = (y_val - o2) / d2 + 0.5
            
            # Check for out-of-bounds indices
            if self._i1[index] < 0 or self._i2[index] < 0 or self._i1[index] >= n1 or self._i2[index] >= n2:
                self._i1[index] = 0
                self._i2[index] = 0
                self._sc[index] = 0

    def forward(self, add, mod, dat):
        """
        Forward operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        for d_val, i1_val, i2_val, sc_val in zip(dat, self._i1, self._i2, self._sc):
            d_val += sc_val * mod[i1_val, i2_val]

    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        for i1_val, i2_val, sc_val, d_val in zip( self._i1, self._i2, self._sc, dat):
            mod[i1_val, i2_val] += sc_val * d_val


## Find the range of x,y 

In [ ]:
xyz=vec.get_nd_array()
x=xyz[:,0]
y=xyz[:,1]
z=xyz[:,2]
print(f"range of x : {x.min()} - {x.max()}")
print(f"range of y : {y.min()} - {y.max()}")
print(f"range of z : {z.min()} - {z.max()}")

## Define our regular grid

In [ ]:
n1=200
n2=200
o1=198
o2=234
d1=(213-o1)/n1
d2=(256-234)/n2

## Create our model space

In [ ]:
from sep_python import Axis,Hypercube, get_sep_vector
ax1=Axis(n=n1,o=o1,d=d1)
ax2=Axis(n=n2,o=o2,d=d2)
hyper=Hypercube([ax1,ax2])
mod=get_sep_vector(hyper)

In [ ]:
# Create our data 

In [ ]:
dat=get_sep_vector(z)

# Hessian approximation 
Create a function that approximates the inverse hessian 

In [ ]:
def diagonal_approx(oper):
    dat=oper.range.clone()
    mod=oper.domain.clone()
    mod.zero()
    dat[:]=1
    oper.adjoint(False,mod,dat)
    return np.where(mod.get_nd_array() !=0, 1./mod.get_nd_array(), 0)


## Form our operator

In [ ]:
import numpy as np
xy=np.vstack((x,y))
bin_op=bin_2d(mod,dat,xy)

In [ ]:
## Approximate the inverse

In [ ]:
mprime=mod.clone()
bin_op.adjoint(False, mprime,dat)
approx=diagonal_approx(bin_op)
mod[:,:]=mprime[:,:]*approx[:,:]

In [ ]:
import sep_plot
sep_plot.Grey(mod,bclip=-250 ,eclip=-200,invert_yaxis=False)